# New Section

In [13]:
!pip install git+https://github.com/fra31/auto-attack

  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-97tkv5vt
  Running command git clone -q https://github.com/fra31/auto-attack /tmp/pip-req-build-97tkv5vt
  Created wheel for autoattack: filename=autoattack-0.1-cp37-none-any.whl size=38060 sha256=01c9321124edc8af716b66746a243e7347e7be75a208875959560b084c04059f
  Stored in directory: /tmp/pip-ephem-wheel-cache-xo5hfv2e/wheels/8e/31/85/1b83bdb1fd788641d8af4e9fac18294a9a6d38a789f82a3e66
Successfully built autoattack


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

In [15]:
from autoattack import AutoAttack


In [17]:
import torchvision.models as models
model = models.resnet18()


In [18]:

!wget "https://github.com/fra31/auto-attack/blob/master/autoattack/examples/model_test.pt?raw=true"

--2021-05-11 18:39:12--  https://github.com/fra31/auto-attack/blob/master/autoattack/examples/model_test.pt?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/fra31/auto-attack/raw/master/autoattack/examples/model_test.pt [following]
--2021-05-11 18:39:13--  https://github.com/fra31/auto-attack/raw/master/autoattack/examples/model_test.pt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fra31/auto-attack/master/autoattack/examples/model_test.pt [following]
--2021-05-11 18:39:13--  https://raw.githubusercontent.com/fra31/auto-attack/master/autoattack/examples/model_test.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githu

In [19]:
# import pickle as pkl
# t = open("/content/model_test.pt",'rb')
# temp = pkl.load(t)
# torch.save()
model = ResNet18()
ckpt = torch.load("/content/model_test.pt?raw=true")
model.load_state_dict(ckpt)
model.cuda()
model.eval()

RuntimeError: ignored

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())


In [ ]:
adversary = AutoAttack(model)
class args():
  def __init__(self):
    self.data_dir = './data'
    self.norm = 'Linf'
    self.epsilon = 8./255.
    self.model = '/content/model_test.pt?raw=true'
    self.n_ex = 1000
    self.individual = 'store_false'
    self.save_dir = './results'
    self.batch_size = 500
    self.log_path = './log_file.txt'
    self.version = 'standard' 
  
args = args()

setting parameters for standard version


In [8]:
import os
import argparse
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms

import sys

In [9]:
# load model
model = ResNet18()
ckpt = torch.load(args.model)
model.load_state_dict(ckpt)
model.cuda()
model.eval()

# load data
transform_list = [transforms.ToTensor()]
transform_chain = transforms.Compose(transform_list)
item = datasets.CIFAR10(root=args.data_dir, train=False, transform=transform_chain, download=True)
test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)

# create save dir
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

# load attack    
from autoattack import AutoAttack
adversary = AutoAttack(model, norm=args.norm, eps=args.epsilon, log_path=args.log_path,
    version=args.version)

l = [x for (x, y) in test_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in test_loader]
y_test = torch.cat(l, 0)

# example of custom version
if args.version == 'custom':
    adversary.attacks_to_run = ['apgd-ce', 'fab']
    adversary.apgd.n_restarts = 2
    adversary.fab.n_restarts = 2



NameError: ignored

In [ ]:
# run attack and save images
with torch.no_grad():
    if not args.individual:
        adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
            bs=args.batch_size)
        
        # torch.save({'adv_complete': adv_complete}, '{}/{}_{}_1_{}_eps_{:.5f}.pth'.format(
        #     args.save_dir, 'aa', args.version, adv_complete.shape[0], args.epsilon))

    else:
        # individual version, each attack is run on all test points
        adv_complete = adversary.run_standard_evaluation_individual(x_test[:args.n_ex],
            y_test[:args.n_ex], bs=args.batch_size)
        
        # torch.save(adv_complete, '{}/{}_{}_individual_1_{}_eps_{:.5f}_plus_{}_cheap_{}.pth'.format(
        #     args.save_dir, 'aa', args.version, args.n_ex, args.epsilon))

using standard version including apgd-ce, apgd-t, fab-t, square
robust accuracy by APGD-CE 	 0.00% 	 (time attack: 114.4 s)
robust accuracy by APGD-T 	 0.00% 	 (time attack: 118.4 s)
robust accuracy by FAB-T 	 0.00% 	 (time attack: 143.3 s)
robust accuracy by SQUARE 	 0.10% 	 (time attack: 131.0 s)


In [ ]:
# run attack and save images
with torch.no_grad():
    if not args.individual:
        # adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
        #     bs=args.batch_size)
        
        torch.save({'adv_complete': adv_complete}, '{}/{}_{}_1_{}_eps_{:.5f}.pth'.format(
            args.save_dir, 'aa', args.version, adv_complete.shape[0], args.epsilon))

    else:
        # individual version, each attack is run on all test points
        # adv_complete = adversary.run_standard_evaluation_individual(x_test[:args.n_ex],
        #     y_test[:args.n_ex], bs=args.batch_size)
        
        torch.save(adv_complete, '{}/{}_{}_individual_1_{}_eps_{:.5f}_plus_cheap.pth'.format(
            args.save_dir, 'aa', args.version, args.n_ex, args.epsilon))